In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys
sys.path.append('../')

## Visualize the predictions


In [6]:
from utils.visualization.vis_utils import VisFromLabelMeToImg

In [7]:
experiment_name = 'singletask_resnet_fpn_verbtarget_parallel_fc'
img_dir = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/img_dir"
pred_ann_dir = f'../resnet_model/work_dirs/{experiment_name}/combine_first_and_second_stage_results' 
pred_store_vis_dir = f'../resnet_model/work_dirs/{experiment_name}/vis_dir_pred'


vis_for_reannotation_generator = VisFromLabelMeToImg(img_dir=img_dir,
                                                    ann_dir=pred_ann_dir,
                                                    vis_dir=pred_store_vis_dir)

In [8]:
vis_for_reannotation_generator.run(make_video=False,
                                   show_img_name=False,
                                   vid_shape = (854, 480))

## Visualize the GT

In [13]:
experiment_name = 'singletask_resnet_fpn_verbtarget_parallel_fc'
img_dir = "../data/triplet_segmentation_dataset_v2_second_stage/mask2former_instrument_prediction/img_dir"
gt_ann_dir = "../data/triplet_segmentation_dataset_v2_second_stage/test/ann_dir"
gt_store_vis_dir = f'../resnet_model/work_dirs/{experiment_name}/vis_gt'


vis_for_reannotation_generator = VisFromLabelMeToImg(img_dir=img_dir,
                                                    ann_dir=gt_ann_dir,
                                                    vis_dir=gt_store_vis_dir)

In [14]:
vis_for_reannotation_generator.run(make_video=False,
                                   show_img_name=False,
                                   vid_shape = (854, 480))

### Combine the folders

In [17]:
import os
from PIL import Image, ImageDraw, ImageFont

def combine_images(pred_vis_dir, gt_dir, output_dir, header_height=40, font_size=30):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # List of files assuming all directories have the same filenames
    filenames = os.listdir(pred_vis_dir)
    
    # Set font for headers
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()
    
    for filename in filenames:
        # Load images
        pred_img = Image.open(os.path.join(pred_vis_dir, filename))
        gt_img = Image.open(os.path.join(gt_dir, filename))
        
        # Determine dimensions
        width, height = pred_img.size
        total_width = width * 2
        total_height = height + header_height
        
        # Create a new blank image
        combined_img = Image.new('RGB', (total_width, total_height), (255, 255, 255))
        draw = ImageDraw.Draw(combined_img)
        
        # Add headers
        draw.rectangle([0, 0, width, header_height], fill=(255, 255, 255))
        draw.rectangle([width, 0, width * 2, header_height], fill=(255, 255, 255))
        
        # Calculate text positions
        pred_text_box = draw.textbbox((0, 0), "Pred", font=font)
        maskrcnn_text_width = pred_text_box[2] - pred_text_box[0]
        maskrcnn_text_height = pred_text_box[3] - pred_text_box[1]
        
        
        gt_text_bbox = draw.textbbox((0, 0), "Ground Truth", font=font)
        gt_text_width = gt_text_bbox[2] - gt_text_bbox[0]
        gt_text_height = gt_text_bbox[3] - gt_text_bbox[1]
        
        # Draw headers
        draw.text((width // 2 - maskrcnn_text_width // 2, header_height // 2 - maskrcnn_text_height // 2),
                  "Pred", font=font, fill=(0, 0, 0))
        draw.text((width + width // 2 - gt_text_width // 2, header_height // 2 - gt_text_height // 2),
                  "Ground Truth", font=font, fill=(0, 0, 0))
        
        
        # Paste images below headers
        combined_img.paste(pred_img, (0, header_height))
        combined_img.paste(gt_img, (width, header_height))
        
        # Save the combined image
        combined_img.save(os.path.join(output_dir, filename))




In [18]:
# Example usage
experiment_name = 'singletask_resnet_fpn_verbtarget_parallel_fc'
combined_store_vis_dir = f'../resnet_model/work_dirs/{experiment_name}/combined'

combine_images(pred_store_vis_dir, gt_store_vis_dir, combined_store_vis_dir)

## Make the videos

In [12]:
from utils.visualization.video_utils import visualize_as_videos
from os.path import join

In [13]:
parent_dir = os.path.dirname(combined_store_vis_dir)
video_output_path =  join(parent_dir, 'combined_video.mp4') 
vid_shape = (1708, 520)

In [15]:
visualize_as_videos(img_dir=img_dir, 
                    video_output_path=video_output_path, 
                    vid_shape=vid_shape,
                    video_fps=1)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2562, 520) to (2576, 528) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


../../results/combined_videos/sparse/t80_VID67_000000.png
../../results/combined_videos/sparse/t80_VID67_000030.png
../../results/combined_videos/sparse/t80_VID67_000060.png
../../results/combined_videos/sparse/t80_VID67_000090.png
../../results/combined_videos/sparse/t80_VID67_000120.png
../../results/combined_videos/sparse/t80_VID67_000150.png
../../results/combined_videos/sparse/t80_VID67_000180.png
../../results/combined_videos/sparse/t80_VID67_000210.png
../../results/combined_videos/sparse/t80_VID67_000240.png
../../results/combined_videos/sparse/t80_VID67_000270.png
../../results/combined_videos/sparse/t80_VID67_000300.png
../../results/combined_videos/sparse/t80_VID67_000330.png
../../results/combined_videos/sparse/t80_VID67_000360.png
../../results/combined_videos/sparse/t80_VID67_000390.png
../../results/combined_videos/sparse/t80_VID67_000420.png
../../results/combined_videos/sparse/t80_VID67_000450.png
../../results/combined_videos/sparse/t80_VID67_000480.png
../../results/

## Creating a movie

full - 0 to 0:20, 2:30 - 3:00,  4:30 - 4:50 , 5:39 - 6:20,  
sparse - 0:04-0:20, 0:35-0:45,  1:29-1:45

In [16]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, ColorClip

# Define the paths to your videos
full_video_path = '../../results/combined_videos/full.mp4'
sparse_video_path = '../../results/combined_videos/sparse.mp4'

# Load the video clips
full_video = VideoFileClip(full_video_path)
sparse_video = VideoFileClip(sparse_video_path)

# Define the segments to cut from each video
full_segments = [
    (0, 20),
    (150, 180),
    (270, 290),
    (339, 380)
]

sparse_segments = [
    (4, 20),
    (35, 45),
    (89, 105)
]

# Function to create a black screen clip for pause
def create_pause_clip(duration, size=(1920, 1080)):
    return ColorClip(size, color=(0, 0, 0), duration=duration)

# Create clips for each segment with pauses
clips = []

for start, end in full_segments:
    clip = full_video.subclip(start, end)
    clips.append(clip)
    clips.append(create_pause_clip(1, size=clip.size))  # Adding 1 second pause

for start, end in sparse_segments:
    clip = sparse_video.subclip(start, end)
    clips.append(clip)
    clips.append(create_pause_clip(1, size=clip.size))  # Adding 1 second pause

# Remove the last pause clip
if clips[-1].duration == 1:
    clips = clips[:-1]

# Concatenate the clips
final_video = concatenate_videoclips(clips, method="compose")

# Define the path for the output video
output_path = '../../results/combined_videos/combined.mp4'

# Write the result to a file
final_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

print("Video has been successfully created and saved.")


[autoreload of decorator failed: Traceback (most recent call last):
  File "c:\Users\tal22\Documents\repositories\cholec_instance_seg\.cholecinstanceseg\lib\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\tal22\Documents\repositories\cholec_instance_seg\.cholecinstanceseg\lib\site-packages\IPython\extensions\autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\tal22\Documents\repositories\cholec_instance_seg\.cholecinstanceseg\lib\site-packages\IPython\extensions\autoreload.py", line 393, in update_generic
    update(a, b)
  File "c:\Users\tal22\Documents\repositories\cholec_instance_seg\.cholecinstanceseg\lib\site-packages\IPython\extensions\autoreload.py", line 305, in update_function
    setattr(old, name, getattr(new, name))
ValueError: ContextManager() requires a code object with 3 free vars, not 0
]


The history saving thread hit an unexpected error (NameError("name 'fix' is not defined")).History will not be written to the database.
Moviepy - Building video ../../results/combined_videos/combined.mp4.
Moviepy - Writing video ../../results/combined_videos/combined.mp4



Moviepy - Done !
Moviepy - video ready ../../results/combined_videos/combined.mp4
Video has been successfully created and saved.
